# Лекция 11: модели в Django

Для абстрагирования при работе с данными в Django используется понятие слоя моделей (model layer).

## Django ORM

* Для взаимодействия с базой в Django принято использовать встроенный ORM (Object-Relational Mapper, механизм объектно-ориентированного отображения).
* Описание объектное представление данных задаётся с помощью моделей (models).
* Модель отображается на отдельную таблицу в базе данных.
* Фреймворк представляет автоматически генерируемое API для работы с данными и генерации операций при работе с базой.
* Django ORM имеет встроенный механизм миграций.


## Модели

* Модель - класс наследник *django.db.models.Model*.
* Атрибут модели - поле в базе данных.
* По этим данным Django ORM автоматически генерирует API для работы с базой.
* При работе с моделями нужно использовать миграции и не забывать их генерировать\применять при изменении моделей.

## Атрибуты моделей

* Поля класса описывающего модель.
* Наследники django.db.models.Field.
* С помощью их определяется:
  * тип данных в базе
  * HTML виджет, который будет использован при рендеринге формы
  * базовые примитивные правила валидации
* Есть много уже готовых классов (примеры):
  * IntegerField
  * BooleanField
  * CharField
  * DateField
  * DurationField
  * ImageField
  * и т.д.
* Специальный атрибут objects, создаётся для каждой модели автоматически и является интерфейсом через который происходит взаимодействие с базой.

## Параметры аттрибутов
  * Есть как свои для отдельных типов полей, так и общие.
  * Некоторые общие аргументы (опциональны):
    * null - если True, то NULL для пустых значений в базе.
    * blank - если True, то поле может быть пустым (с точки зрения валидации, а не данных в базе).
    * choices - iterable и пар (значение\_в\_базе, человекочитаемый\_текст).
    * default - значение по-умолчанию для поля.
    * help_text - текст, будет отображен в виджете формы (также удобно просто для документации).
    * unique - значение должно быть уникальным (среди всех таких полей в таблице).
    * primary_key - если True, то текущее поле станет primary ключом в таблице.

Если в таблице модели нет атрибутов с параметром primary_key, то отдельное поле будет создано автоматически.

In [ ]:
id = models.AutoField(primary_key=True)

## Примеры моделей

In [ ]:
from django.db import models

class Musician(models.Model):
    first_name = models.CharField(max_length=50)
    last_name = models.CharField(max_length=50)
    instrument = models.CharField(max_length=100)

class Album(models.Model):
    artist = models.ForeignKey(Musician)
    name = models.CharField(max_length=100)
    release_date = models.DateField()
    num_stars = models.IntegerField()

In [ ]:
from django.db import models

class Fruit(models.Model):
    name = models.CharField(max_length=100, primary_key=True)

In [ ]:
>>> fruit = Fruit.objects.create(name='Apple')
>>> fruit.name = 'Pear'
>>> fruit.save()
>>> Fruit.objects.values_list('name', flat=True)
['Apple', 'Pear']

In [ ]:
from django.db import models

class Person(models.Model):
    name = models.CharField(max_length=128)

    def __str__(self):              # __unicode__ on Python 2
        return self.name

class Group(models.Model):
    name = models.CharField(max_length=128)
    members = models.ManyToManyField(Person, through='Membership')

    def __str__(self):              # __unicode__ on Python 2
        return self.name

class Membership(models.Model):
    person = models.ForeignKey(Person)
    group = models.ForeignKey(Group)
    date_joined = models.DateField()
    invite_reason = models.CharField(max_length=64)

In [ ]:
>>> ringo = Person.objects.create(name="Ringo Starr")
>>> paul = Person.objects.create(name="Paul McCartney")
>>> beatles = Group.objects.create(name="The Beatles")
>>> m1 = Membership(person=ringo, group=beatles,
...     date_joined=date(1962, 8, 16),
...     invite_reason="Needed a new drummer.")
>>> m1.save()
>>> beatles.members.all()
[<Person: Ringo Starr>]
>>> ringo.group_set.all()
[<Group: The Beatles>]
>>> m2 = Membership.objects.create(person=paul, group=beatles,
...     date_joined=date(1960, 8, 1),
...     invite_reason="Wanted to form a band.")
>>> beatles.members.all()
[<Person: Ringo Starr>, <Person: Paul McCartney>]

Подробнее про модели:  
* https://docs.djangoproject.com/en/1.8/#the-model-layer

## Практика

Рассмотрим первую главу туториала (+вопросы):
  * https://docs.djangoproject.com/en/1.8/intro/tutorial01/
  * http://djbook.ru/rel1.8/intro/tutorial01.html